In [33]:
!pip install lightgbm
!pip install scikit-optimize

     |████████████████████████████████| 101 kB 979 kB/s ta 0:00:011


In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
df = pd.read_csv("df_model3.csv", index_col=0).dropna(subset=["y"])
df

,title,upload_date,view_count,tempo_desde_pub,y
318,Mindset /w Killa Keim (From DownTown) | BBOY.O...,2021-03-11,4728,201.0,0.0
319,BBOY ZOOPREME | MUSIC IN HIS SOUL 🎶,2021-09-22,1356,6.0,1.0
323,BBOY ZOOTY ZOOT | AMAZING FLOW 🌊,2021-09-14,2902,14.0,1.0
326,Bboy music 2021- run it,2021-10-02,43,-4.0,0.0
327,Red Bull BC One B-Boy Cypher Japan 2021 | LIVE...,2021-09-26,50041,2.0,1.0
...,...,...,...,...,...
302,Breakdance Top 14 Suicide Moves 2019 || Best ...,2020-04-08,1910545,538.0,0.0
303,Chris Brown -- The INSANE Breakdance Battle | TMZ,2011-12-06,655891,3584.0,0.0
304,Lazy legs Disabled breakdancer,2006-07-22,319703,5547.0,0.0
305,ITZY “LOCO” Dance Tutorial | Chorus + Dance Br...,2021-09-24,170975,4.0,0.0


In [4]:
df.duplicated().mean()

0.014571948998178506

In [5]:
df.duplicated(['title']).mean()

0.023679417122040074

In [6]:
df.shape

(549, 5)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [8]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']


## 1. Limpeza da data

In [9]:

df_limpo['date'] = pd.to_datetime(df['upload_date'])

## 2. Limpeza de Views

In [10]:
df_limpo['views'] = df['view_count']

## 3. Features

In [11]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [12]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [13]:
features.head()

,views,views_por_dia
318,4728,-10.189655
319,1356,-2.057663
323,2902,-4.457757
326,43,-0.064275
327,50041,-75.476621


In [14]:
mask_train = df_limpo['date'] < "2019-04-01"
mask_val = (df_limpo['date'] >= "2019-04-01")

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((178, 2), (371, 2), (178,), (371,))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2,ngram_range=(1,4))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)


intro to machine learning -> intro, to, machine, learning  -> ngram_range=(1,1)  
intro to machine learning -> intro, to, machine, learning, intro to, to machine, machine learning -> ngram_range=(1,2)  
intro to machine learning -> intro to, to machine, machine learning -> ngram_range=(2,2)  


In [16]:
title_bow_train.shape

(178, 394)

In [17]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [18]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((178, 396), (371, 396))

# 4 Random Forest

In [19]:
mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=2, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(class_weight='balanced', min_samples_leaf=2,
                       n_estimators=1000, n_jobs=6, random_state=0)

In [20]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [21]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [22]:
average_precision_score(yval, p)

0.49782259437866505

In [23]:
roc_auc_score(yval, p)

0.7221044484824013

ap 0.5009690496298342 auc 0.7260246315364427, min_samples_leaf=1, mind_df=1, n_estimators=1000
ap 0.5041640609074963 auc 0.721818426542836 min_samples_leaf=1, mind_df=2, n_estimators=1000

ap 0.5041640609074963 auc 0.721818426542836 min_samples_leaf=1,  n_estimators=1000, mind_df=2, ngram_range=(1,2)
ap 0.5062122233234501 auc 0.7262265293761356  min_samples_leaf=1,  n_estimators=1000, mind_df=2, ngram_range=(1,3)
ap 0.49782259437866505 auc 0.7221044484824013  min_samples_leaf=1,  n_estimators=1000, mind_df=2, ngram_range=(1,4)


ap 0.4909543267817009 auc 0.7183356888081297 min_samples_leaf=1, mind_df=2, n_estimators=50
ap 0.489915631175127 auc 0.7218352513628105  min_samples_leaf=2, mind_df=2, n_estimators=50
ap .4563569555419245 auc 0.7118917827579245  min_samples_leaf=2, mind_df=2, n_estimators=50



# 5 LightGBM

In [27]:
from lightgbm import LGBMClassifier

In [28]:
mdl = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(class_weight='balanced', n_jobs=6, random_state=0)

In [29]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

/home/kevintakano/miniconda3/lib/python3.8/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [30]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.3761307419412462, 0.5984083720304193)

# 6 Bayesian Optimization

In [34]:
from skopt import forest_minimize

In [35]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    
    Xtrain_wtitle = hstack([Xtrain, title_bow_train])
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=6)
    mdl.fit(Xtrain_wtitle, ytrain)
    
    p = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval, p)


space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000), # n_estimators
          (1,5), # min_df
          (1,5)] # ngram_range

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.6888081297530116
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.1339
Function value obtained: -0.4730
Current minimum: -0.4730
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.6255636314691433
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0620
Function value obtained: -0.3831
Current minimum: -0.4730
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]


/home/kevintakano/miniconda3/lib/python3.8/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/home/kevintakano/miniconda3/lib/python3.8/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/home/kevintakano/miniconda3/lib/python3.8/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.6052897233999596
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0728
Function value obtained: -0.3633
Current minimum: -0.4730
Iteration No: 4 started. Evaluating function at random point.
[0.0014099928811969545, 9, 9, 0.6502182010234373, 0.6866210554187129, 828, 5, 2]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


/home/kevintakano/miniconda3/lib/python3.8/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/home/kevintakano/miniconda3/lib/python3.8/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.671966484958611
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.2003
Function value obtained: -0.4464
Current minimum: -0.4730
Iteration No: 5 started. Evaluating function at random point.
[0.08530558241838007, 8, 19, 0.2137736299768322, 0.1313765544201984, 961, 4, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.6356416986338245
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0598
Function value obtained: -0.4011
Current minimum: -0.4730
Iteration No: 6 started. Evaluating function at random point.
[0.003567949451535685, 10, 19, 0.7232951768944309, 0.7298538828427115, 939, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


...

Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 0.4247
Function value obtained: -0.4568
Current minimum: -0.5041
Iteration No: 49 started. Searching for the next optimal point.
[0.006001246525392472, 10, 2, 0.9772058266680003, 0.5358040181839454, 920, 3, 5]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


/home/kevintakano/miniconda3/lib/python3.8/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.7026213069520156
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 0.9820
Function value obtained: -0.4822
Current minimum: -0.5041
Iteration No: 50 started. Searching for the next optimal point.
[0.0046479364930115605, 7, 1, 0.9184968201766729, 0.4811008423812129, 824, 1, 5]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


/home/kevintakano/miniconda3/lib/python3.8/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


0.6947136415640353
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 1.7136
Function value obtained: -0.4519
Current minimum: -0.5041


ap 0.22228951304206077, auc 0.6914990859232175 - min_df=2, ngram_range=(1,3) RF 
ap 0.2378, auc 0.6883293035324645 LGBM - [0.08265121231498246, 7, 1, 0.7251351011494334, 0.07547006552546137, 839, 2, 5]


In [36]:
res.x

[0.0040495256441752305,
 6,
 2,
 0.9181734790929957,
 0.44546924957870876,
 931,
 2,
 5]

# 7 Logistic Reg

In [109]:
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [115]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
scaler = MaxAbsScaler()


#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
Xval_wtitle2 = scaler.transform(Xval_wtitle2)

In [116]:
Xval_wtitle2.shape

(609, 1335)

In [135]:

mdl = LogisticRegression(C=0.5,n_jobs=6, random_state=0)
mdl.fit(Xtrain_wtitle2, ytrain)

/Users/mario/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/mario/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=6,
          penalty='l2', random_state=0, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [136]:
p = mdl.predict_proba(Xval_wtitle2)[:, 1]

In [137]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.21340786207179874, 0.6835525151854692)

In [ ]:
(0.20616279322296893, 0.6606416229285841) - sem tuning, standardscaler

(0.20757989629841797, 0.6862357728371764) - sem tuning, maxabsscaler
(0.18953863996413214, 0.6741463702305833) - C=10, maxabsscaler

(0.21340786207179874, 0.6835525151854692) - C=0.5, maxabsscaler